In [1]:
import cobra
from cobra import Model, Reaction, Metabolite
from cobra.flux_analysis import pfba
from cobra.io import write_sbml_model,save_json_model,load_json_model,read_sbml_model
import re
import pandas as pd
import numpy as np
import xlrd
import re
import openpyxl
import sys
sys.path.append(r'./')
import os
from copy import copy, deepcopy
import numpy as np

In [2]:
model_origin_name='./new_idea_AGGD/model/iML1515_new.json'
model_new='./new_idea_AGGD/model/iML1515_AG.json'
model_new2='./new_idea_AGGD/model/iML1515_GD.json'
model_new3='./new_idea_AGGD/model/iML1515_GD_26.json'
model_ab_json='./new_idea_AGGD/model/iML1515_AGGD.json'

# Refinement of model information

In [3]:
from model_correction_tools import *

In [4]:
model = cobra.io.load_json_model(model_origin_name)
model.reactions.EX_tyr__L_e.bounds=(0,1000)
save_json_model(model,model_origin_name)

sub-model A

In [5]:
model = cobra.io.load_json_model(model_origin_name)

In [6]:
model.reactions.get_by_id('EX_xyl__D_e').bounds=(-10,1000)
model.reactions.get_by_id('EX_glc__D_e').bounds=(0,1000)

In [7]:
#ptsG/manZ
model.reactions.get_by_id('GLCptspp')#.bounds=(0,0)

Reaction identifier,GLCptspp
Name,D-glucose transport via PEP:Pyr PTS (periplasm)
Memory address,0x027ea0bd82b0
Stoichiometry,glc__D_p + pep_c --> g6p_c + pyr_c D-Glucose + Phosphoenolpyruvate --> D-Glucose 6-phosphate + Pyruvate
GPR,( b2415 and b1101 and b2416 and b2417 ) or ( b2417 and b1621 and b2415 and b2416 ) or ( b1817 and...
Lower bound,0.0
Upper bound,1000.0


In [8]:
#PYK
PYK=['PYK','PYK2','PYK3','PYK4','PYK6']
for i in PYK:
    print(model.reactions.get_by_id(i))
    model.reactions.get_by_id(i).bounds=(0,0)

PYK: adp_c + h_c + pep_c --> atp_c + pyr_c
PYK2: h_c + pep_c + udp_c --> pyr_c + utp_c
PYK3: gdp_c + h_c + pep_c --> gtp_c + pyr_c
PYK4: cdp_c + h_c + pep_c --> ctp_c + pyr_c
PYK6: dtdp_c + h_c + pep_c --> dttp_c + pyr_c


In [9]:
#pheA PPNDH
print(model.reactions.get_by_id('PPNDH'))
model.reactions.get_by_id('PPNDH').bounds=(0,0)
#feaB ALDD19xr NO KNOCK
print(model.reactions.get_by_id('ALDD19xr'))
model.reactions.get_by_id('ALDD19xr').bounds=(-1000,1000)

PPNDH: h_c + pphn_c --> co2_c + h2o_c + phpyr_c
ALDD19xr: h2o_c + nad_c + pacald_c <=> 2.0 h_c + nadh_c + pac_c


In [10]:
#tyrA PPMD
print(model.reactions.get_by_id('PPND'))
model.reactions.get_by_id('PPND')

PPND: nad_c + pphn_c --> 34hpp_c + co2_c + nadh_c


Reaction identifier,PPND
Name,Prephenate dehydrogenase
Memory address,0x027ea07bee80
Stoichiometry,nad_c + pphn_c --> 34hpp_c + co2_c + nadh_c Nicotinamide adenine dinucleotide + Prephenate --> 3-(4-Hydroxyphenyl)pyruvate + CO2 CO2 + Nicotinamide adenine dinucleotide - reduced
GPR,b2600
Lower bound,0.0
Upper bound,1000.0


In [11]:
#ACMANAptspp manZ
print(model.reactions.get_by_id('ACMANAptspp'))
model.reactions.get_by_id('ACMANAptspp').bounds=(0,0)

ACMANAptspp: acmana_p + pep_c --> acmanap_c + pyr_c


In [12]:
#MANptspp manZ
print(model.reactions.get_by_id('MANptspp'))
model.reactions.get_by_id('MANptspp').bounds=(0,0)

MANptspp: man_p + pep_c --> man6p_c + pyr_c


In [13]:
#FRUpts2pp manZ
print(model.reactions.get_by_id('FRUpts2pp'))
model.reactions.get_by_id('FRUpts2pp').bounds=(0,0)

FRUpts2pp: fru_p + pep_c --> f6p_c + pyr_c


In [14]:
#2DGLCptspp manZ
print(model.reactions.get_by_id('2DGLCptspp'))
model.reactions.get_by_id('2DGLCptspp').bounds=(0,0)

2DGLCptspp: 2dglc_p + pep_c --> 2doxg6p_c + pyr_c


In [15]:
hoxpacd_c=deepcopy(model.metabolites.get_by_id('4hoxpacd_e'))
hoxpacd_c.compartment='c'
hoxpacd_c.id='hoxpacd_c'
hoxpacd_c

Metabolite identifier,hoxpacd_c
Name,4-Hydroxyphenylacetaldehyde
Memory address,0x027e9e2a5550
Formula,C8H8O2
Compartment,c
In 0 reaction(s),


In [16]:
hpp_c=model.metabolites.get_by_id('34hpp_c')
h_c=model.metabolites.get_by_id('h_c')
co2_c=model.metabolites.get_by_id('co2_c')
PDC=Reaction("PDC")
PDC.name='pyruvate decarboxylase'
PDC.subsystem='unknow'
PDC.lower_bound=0
PDC.upper_bound=1000
PDC.add_metabolites({
    hpp_c:-1.0,
    h_c:-1.0,
    co2_c:1.0,
    hoxpacd_c:1.0
})
PDC

Reaction identifier,PDC
Name,pyruvate decarboxylase
Memory address,0x027e9e246a58
Stoichiometry,34hpp_c + h_c --> co2_c + hoxpacd_c 3-(4-Hydroxyphenyl)pyruvate + H+ --> CO2 CO2 + 4-Hydroxyphenylacetaldehyde
GPR,
Lower bound,0
Upper bound,1000


In [17]:
Tyrosol_c=Metabolite('Tyrosol_c',
                formula='C8H10O2',
                name='Tyrosol',
                compartment='c'
                )

In [18]:
Tyrosol_e=Metabolite('Tyrosol_e',
                formula='C8H10O2',
                name='Tyrosol',
                compartment='e'
                )

In [19]:
EX_Tyrosol_e=Reaction("EX_Tyrosol_e")
EX_Tyrosol_e.name='Tyrosol_e exchange'
EX_Tyrosol_e.subsystem='unknow'
EX_Tyrosol_e.lower_bound=0
EX_Tyrosol_e.upper_bound=1000
EX_Tyrosol_e.add_metabolites({
    Tyrosol_e:-1.0
})
Tyrosol_c2e=Reaction("Tyrosol_c2e")
Tyrosol_c2e.name='4adcho_c2e'
Tyrosol_c2e.subsystem='unknow'
Tyrosol_c2e.lower_bound=-1000
Tyrosol_c2e.upper_bound=1000
Tyrosol_c2e.add_metabolites({
    Tyrosol_c:-1.0,
    Tyrosol_e:1.0
})
model.add_reactions([EX_Tyrosol_e])
model.add_reactions([Tyrosol_c2e])

In [20]:
phpyr_c=model.metabolites.get_by_id('phpyr_c')
phpyr_e=deepcopy(phpyr_c)
phpyr_e.compartment='e'
phpyr_e.id='phpyr_e'
phpyr_e

Metabolite identifier,phpyr_e
Name,Phenylpyruvate
Memory address,0x027e9e2a5828
Formula,C9H7O3
Compartment,e
In 0 reaction(s),


In [21]:
EX_phpyr_e=Reaction("EX_phpyr_e")
EX_phpyr_e.name='phpyr_e exchange'
EX_phpyr_e.subsystem='unknow'
EX_phpyr_e.lower_bound=-1000
EX_phpyr_e.upper_bound=1000
EX_phpyr_e.add_metabolites({
    phpyr_e:-1.0
})
phpyr_c2e=Reaction("phpyr_c2e")
phpyr_c2e.name='phpyr_c2e'
phpyr_c2e.subsystem='unknow'
phpyr_c2e.lower_bound=-1000
phpyr_c2e.upper_bound=1000
phpyr_c2e.add_metabolites({
    phpyr_c:-1.0,
    phpyr_e:1.0
})
model.add_reactions([EX_phpyr_e])
model.add_reactions([phpyr_c2e])

In [22]:
nad_c=model.metabolites.get_by_id('nad_c')
h_c=model.metabolites.get_by_id('h_c')
nadh_c=model.metabolites.get_by_id('nadh_c')
ADH=Reaction("ADH")
ADH.name='alcohol dehydrogenase'
ADH.subsystem='unknow'
ADH.lower_bound=0
ADH.upper_bound=1000
ADH.add_metabolites({
    hoxpacd_c:-1.0,
    nadh_c:-1.0,
    h_c:-1.0,
    nad_c:1.0,
    Tyrosol_c:1.0
})
ADH

Reaction identifier,ADH
Name,alcohol dehydrogenase
Memory address,0x027e9e2bdc18
Stoichiometry,h_c + hoxpacd_c + nadh_c --> Tyrosol_c + nad_c H+ + 4-Hydroxyphenylacetaldehyde + Nicotinamide adenine dinucleotide - reduced --> Tyrosol + Nicotinamide adenine dinucleotide
GPR,
Lower bound,0
Upper bound,1000


In [23]:
model.add_reactions([PDC])
model.add_reactions([ADH])

In [24]:
with model as model_test:
    product = model_test.metabolites.get_by_id('Tyrosol_c')
    demand= model_test.add_boundary(product, type='demand')  # add demand reaction as the objective
    model_test.objective = demand
    model_test.reactions.BIOMASS_Ec_iML1515_core_75p37M.bounds=(0.1,1000)
    model_test.reactions.EX_tyr__L_e.bounds=(0.1,1000)
    model_test.optimize()
    model_test.summary()

IN FLUXES            OUT FLUXES      OBJECTIVES
-------------------  --------------  ------------------
xyl__D_e  10         h2o_e     28.4  DM_Tyrosol_c  3.96
o2_e       7.26      co2_e     13.5
nh4_e      1.18      h_e        1
pi_e       0.0965    tyr__L_e   0.1
so4_e      0.0252
k_e        0.0195
phpyr_e    0.0185
fe2_e      0.00161
mg2_e      0.000868
ca2_e      0.000521
cl_e       0.000521
cu2_e      7.09e-05
mn2_e      6.91e-05
zn2_e      3.41e-05
ni2_e      3.23e-05


In [25]:
model.optimize()
model.summary()

IN FLUXES            OUT FLUXES    OBJECTIVES
-------------------  ------------  ----------------------
o2_e      20.4       h2o_e  40.1   BIOMASS_Ec_i...  0.715
xyl__D_e  10         co2_e  21.8
nh4_e      7.72      h_e     6.44
pi_e       0.69
so4_e      0.18
k_e        0.14
phpyr_e    0.132
fe2_e      0.0115
mg2_e      0.0062
ca2_e      0.00372
cl_e       0.00372
cu2_e      0.000507
mn2_e      0.000494
zn2_e      0.000244
ni2_e      0.000231


In [26]:
save_json_model(model,model_new)
write_sbml_model(model,model_new.split('.json')[0]+'.xml')

sub-model B

In [27]:
model = cobra.io.load_json_model(model_origin_name)
model.reactions.get_by_id('EX_glc__D_e').bounds=(-10,1000)
model.reactions.get_by_id('EX_xyl__D_e').bounds=(0,1000)

In [28]:
#xylA XYLT1
print(model.reactions.get_by_id('XYLI1'))
model.reactions.get_by_id('XYLI1').bounds=(0,0)
#ushA UDPGPpp
print(model.reactions.get_by_id('UDPGPpp'))
model.reactions.get_by_id('UDPGPpp').bounds=(0,0)
#tyrA PPND
print(model.reactions.get_by_id('PPND'))
# model.reactions.get_by_id('PPND').bounds=(0,0)
print(model.reactions.get_by_id('CHORM'))
# model.reactions.get_by_id('CHORM').bounds=(0,0)
model.reactions.get_by_id('TYRTA').bounds=(0,1000)
model.reactions.get_by_id('TYRPpp').bounds=(0,0)
model.reactions.get_by_id('TYRtipp')

XYLI1: xyl__D_c <=> xylu__D_c
UDPGPpp: h2o_p + udpg_p --> g1p_p + 2.0 h_p + ump_p
PPND: nad_c + pphn_c --> 34hpp_c + co2_c + nadh_c
CHORM: chor_c --> pphn_c


Reaction identifier,TYRtipp
Name,L-tyrosine transport out (periplasm)
Memory address,0x027ea4bfe748
Stoichiometry,tyr__L_c --> tyr__L_p L-Tyrosine --> L-Tyrosine
GPR,b1473
Lower bound,0.0
Upper bound,1000.0


In [29]:
Tyrosol_c=Metabolite('Tyrosol_c',
                formula='C8H10O2',
                name='Tyrosol',
                compartment='c'
                )
Tyrosol_e=Metabolite('Tyrosol_e',
                formula='C8H10O2',
                name='Tyrosol',
                compartment='e'
                )

In [30]:
phpyr_c=model.metabolites.get_by_id('phpyr_c')
phpyr_e=deepcopy(phpyr_c)
phpyr_e.compartment='e'
phpyr_e.id='phpyr_e'
phpyr_e

Metabolite identifier,phpyr_e
Name,Phenylpyruvate
Memory address,0x027e9d705f98
Formula,C9H7O3
Compartment,e
In 0 reaction(s),


In [31]:
EX_phpyr_e=Reaction("EX_phpyr_e")
EX_phpyr_e.name='phpyr_e exchange'
EX_phpyr_e.subsystem='unknow'
EX_phpyr_e.lower_bound=-1000
EX_phpyr_e.upper_bound=1000
EX_phpyr_e.add_metabolites({
    phpyr_e:-1.0
})
phpyr_c2e=Reaction("phpyr_c2e")
phpyr_c2e.name='phpyr_c2e'
phpyr_c2e.subsystem='unknow'
phpyr_c2e.lower_bound=-1000
phpyr_c2e.upper_bound=1000
phpyr_c2e.add_metabolites({
    phpyr_c:-1.0,
    phpyr_e:1.0
})
model.add_reactions([EX_phpyr_e])
model.add_reactions([phpyr_c2e])

In [32]:
EX_Tyrosol_e=Reaction("EX_Tyrosol_e")
EX_Tyrosol_e.name='Tyrosol_e exchange'
EX_Tyrosol_e.subsystem='unknow'
EX_Tyrosol_e.lower_bound=-1000
EX_Tyrosol_e.upper_bound=1000
EX_Tyrosol_e.add_metabolites({
    Tyrosol_e:-1.0
})
Tyrosol_c2e=Reaction("Tyrosol_c2e")
Tyrosol_c2e.name='Tyrosol_c2e'
Tyrosol_c2e.subsystem='unknow'
Tyrosol_c2e.lower_bound=-1000
Tyrosol_c2e.upper_bound=1000
Tyrosol_c2e.add_metabolites({
    Tyrosol_c:-1.0,
    Tyrosol_e:1.0
})
model.add_reactions([EX_Tyrosol_e])
model.add_reactions([Tyrosol_c2e])

In [33]:
Salid_c=Metabolite('Salid_c',
                formula='C14H20O7',
                name='Salidroside',
                compartment='c'
                )

In [34]:
Salid_e=Metabolite('Salid_e',
                formula='C14H20O7',
                name='Salidroside',
                compartment='e'
                )

In [35]:
EX_Salid_e=Reaction("EX_Salid_e")
EX_Salid_e.name='Salid_e exchange'
EX_Salid_e.subsystem='unknow'
EX_Salid_e.lower_bound=0
EX_Salid_e.upper_bound=1000
EX_Salid_e.add_metabolites({
    Salid_e:-1.0
})
Salid_c2e=Reaction("Salid_c2e")
Salid_c2e.name='Salid_c2e'
Salid_c2e.subsystem='unknow'
Salid_c2e.lower_bound=-1000
Salid_c2e.upper_bound=1000
Salid_c2e.add_metabolites({
    Salid_c:-1.0,
    Salid_e:1.0
})
model.add_reactions([EX_Salid_e])
model.add_reactions([Salid_c2e])

In [36]:
udp_c=model.metabolites.get_by_id('udp_c')
udpg_c=model.metabolites.get_by_id('udpg_c')
h_c=model.metabolites.get_by_id('h_c')
RXN_12370=Reaction("RXN_12370")
RXN_12370.name='tyrosol UDP-glucosyltransferase'
RXN_12370.subsystem='unknow'
RXN_12370.lower_bound=0
RXN_12370.upper_bound=1000
RXN_12370.add_metabolites({
    udpg_c:-1.0,
    Tyrosol_c:-1.0,
    h_c:1.0,
    udp_c:1.0,
    Salid_c:1.0
})
RXN_12370

Reaction identifier,RXN_12370
Name,tyrosol UDP-glucosyltransferase
Memory address,0x027e9ecb1208
Stoichiometry,Tyrosol_c + udpg_c --> Salid_c + h_c + udp_c Tyrosol + UDPglucose --> Salidroside + H+ + UDP C9H11N2O12P2
GPR,
Lower bound,0
Upper bound,1000


In [37]:
model.add_reactions([RXN_12370])

In [38]:
model.reactions.get_by_id('EX_glc__D_e').bounds=(-10,1000)
model.reactions.get_by_id('EX_xyl__D_e').bounds=(0,1000)

In [39]:
model.reactions.get_by_id('EX_tyr__L_e').bounds=(-1000,1000)

In [40]:
with model as model_test:
    product2 = model_test.metabolites.get_by_id('Salid_c')
    demand2= model_test.add_boundary(product2, type='demand')  # add demand reaction as the objective
    model_test.objective = demand2
    model_test.reactions.BIOMASS_Ec_iML1515_core_75p37M.bounds=(0.1,1000)
    model_test.reactions.EX_phpyr_e.bounds=(0.8,1000)
    model_test.optimize()
    model_test.summary()

IN FLUXES             OUT FLUXES      OBJECTIVES
--------------------  --------------  ----------------
glc__D_e   10         h2o_e    22.2   DM_Salid_c  6.64
o2_e        8.38      co2_e     8.99
Tyrosol_e   6.64      h_e       1.71
nh4_e       1.07      phpyr_e   0.8
pi_e        0.0965
so4_e       0.0252
k_e         0.0195
tyr__L_e    0.0138
fe2_e       0.00161
mg2_e       0.000868
ca2_e       0.000521
cl_e        0.000521
cu2_e       7.09e-05
mn2_e       6.91e-05
zn2_e       3.41e-05
ni2_e       3.23e-05


In [41]:
model.optimize()
model.summary()

IN FLUXES            OUT FLUXES    OBJECTIVES
-------------------  ------------  ----------------------
o2_e      23.3       h2o_e  47.8   BIOMASS_Ec_i...  0.915
glc__D_e  10         co2_e  25.1
nh4_e      9.75      h_e     8.11
pi_e       0.882
so4_e      0.23
k_e        0.179
phpyr_e    0.169
tyr__L_e   0.126
fe2_e      0.0147
mg2_e      0.00794
cl_e       0.00476
ca2_e      0.00476
cu2_e      0.000649
mn2_e      0.000632
zn2_e      0.000312
ni2_e      0.000295


In [42]:
save_json_model(model,model_new2)
write_sbml_model(model,model_new2.split('.json')[0]+'.xml')
sbml_excel(model_new2.split('.json')[0]+'.xml',model_new2.split('.json')[0]+'.xlsx')

./new_idea_AGGD/model/iML1515_GD.xlsx


### Model Correction

SKMs Construction

In [43]:
model_a_name=model_new
model_b_name=model_new2

In [44]:
model_a=load_json_model(model_a_name)
model_b=load_json_model(model_b_name)

In [45]:
model_a.reactions.get_by_id('EX_glc__D_e').bounds=(0,1000)
model_a.reactions.get_by_id('EX_xyl__D_e').bounds=(-10,1000)
model_a.reactions.get_by_id('EX_phpyr_e').bounds=(-1000,0)
model_b.reactions.get_by_id('EX_Tyrosol_e').bounds=(-1000,0)
model_b.reactions.get_by_id('EX_tyr__L_e').bounds=(-1000,0)
model_b.reactions.get_by_id('EX_xyl__D_e').bounds=(0,1000)
model_b.reactions.get_by_id('EX_glc__D_e').bounds=(-10,1000)

In [46]:
print("--------A  model----------")
print(len(model_a.reactions))
print(len(model_a.metabolites))
print(len(model_a.genes) )
print("--------B  model----------")
print(len(model_b.reactions))
print(len(model_b.metabolites))
print(len(model_b.genes) )

--------A  model----------
2719
1881
1516
--------B  model----------
2720
1882
1516


In [47]:
model=deepcopy(model_a)
for i in model.reactions:
    i.id='A_'+i.id
for i in model.metabolites:
    i.id='A_'+i.id
    i.name='A_'+i.name
model_t=deepcopy(model_b)
# for i in model_t.genes:
#     i.id=i.id+'_B'
for i in model_t.reactions:
    i.id='B_'+i.id
for i in model_t.metabolites:
    i.id='B_'+i.id
    i.name='B_'+i.name

In [48]:
for i in model_t.reactions:
    model.add_reaction(i)

In [49]:
e_meta_rn={}
for meta in model_a.metabolites:
    if meta.compartment=='e' :
        meta_a=model.metabolites.get_by_id('A_'+meta.id)
        e_meta_rn[meta.id+"_1"]=Reaction('A_'+meta.id+'_con_1')
        e_meta_rn[meta.id+"_1"].name = meta.id+' Connection 1'
        e_meta_rn[meta.id+"_1"].subsystem = 'unKnow'
        e_meta_rn[meta.id+"_1"].lower_bound = 0. # This is the default
        e_meta_rn[meta.id+"_1"].upper_bound = 1000. # This is the default
        e_meta_rn[meta.id+"_1"].add_metabolites({
            meta_a:-1.0,
            meta:1.0
            })
        model.add_reactions([e_meta_rn[meta.id+"_1"]])
for meta in model_b.metabolites:
    if meta.compartment=='e' :
        meta_b=model_t.metabolites.get_by_id('B_'+meta.id)
        e_meta_rn[meta.id+"_2"]=Reaction('B_'+meta.id+'_con_2')
        e_meta_rn[meta.id+"_2"].name = meta.id+' Connection 2'
        e_meta_rn[meta.id+"_2"].subsystem = 'unKnow'
        e_meta_rn[meta.id+"_2"].lower_bound = -1000. # This is the default
        e_meta_rn[meta.id+"_2"].upper_bound = 0. # This is the default
        e_meta_rn[meta.id+"_2"].add_metabolites({
            meta_b:1.0,
            meta:-1.0
            })
        model.add_reactions([e_meta_rn[meta.id+"_2"]])

In [50]:
origin_model=cobra.io.load_json_model(model_origin_name)
origin_model.reactions.get_by_id('EX_xyl__D_e').bounds=(-10.0,1000)
origin_model.medium

{'EX_pi_e': 1000.0,
 'EX_h_e': 1000.0,
 'EX_fe3_e': 1000.0,
 'EX_mn2_e': 1000.0,
 'EX_co2_e': 1000.0,
 'EX_fe2_e': 1000.0,
 'EX_glc__D_e': 10.0,
 'EX_xyl__D_e': 10.0,
 'EX_zn2_e': 1000.0,
 'EX_mg2_e': 1000.0,
 'EX_ca2_e': 1000.0,
 'EX_ni2_e': 1000.0,
 'EX_cu2_e': 1000.0,
 'EX_cobalt2_e': 1000.0,
 'EX_sel_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_mobd_e': 1000.0,
 'EX_so4_e': 1000.0,
 'EX_k_e': 1000.0,
 'EX_na1_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_cl_e': 1000.0,
 'EX_tungs_e': 1000.0,
 'EX_slnt_e': 1000.0}

In [51]:
for i in origin_model.medium:
    model.add_reaction(origin_model.reactions.get_by_id(i))
    ex_meta=i.split('EX_')[1]
    model.reactions.get_by_id('A_'+ex_meta+'_con_1').bounds=(-1000,1000)
    model.reactions.get_by_id('B_'+ex_meta+'_con_2').bounds=(-1000,1000)
model.medium

{'A_EX_pi_e': 1000.0,
 'A_EX_h_e': 1000.0,
 'A_EX_fe3_e': 1000.0,
 'A_EX_mn2_e': 1000.0,
 'A_EX_co2_e': 1000.0,
 'A_EX_fe2_e': 1000.0,
 'A_EX_xyl__D_e': 10,
 'A_EX_zn2_e': 1000.0,
 'A_EX_mg2_e': 1000.0,
 'A_EX_ca2_e': 1000.0,
 'A_EX_ni2_e': 1000.0,
 'A_EX_cu2_e': 1000.0,
 'A_EX_cobalt2_e': 1000.0,
 'A_EX_sel_e': 1000.0,
 'A_EX_h2o_e': 1000.0,
 'A_EX_nh4_e': 1000.0,
 'A_EX_mobd_e': 1000.0,
 'A_EX_so4_e': 1000.0,
 'A_EX_k_e': 1000.0,
 'A_EX_na1_e': 1000.0,
 'A_EX_o2_e': 1000.0,
 'A_EX_cl_e': 1000.0,
 'A_EX_tungs_e': 1000.0,
 'A_EX_slnt_e': 1000.0,
 'A_EX_phpyr_e': 1000,
 'B_EX_pi_e': 1000.0,
 'B_EX_h_e': 1000.0,
 'B_EX_fe3_e': 1000.0,
 'B_EX_mn2_e': 1000.0,
 'B_EX_co2_e': 1000.0,
 'B_EX_tyr__L_e': 1000,
 'B_EX_fe2_e': 1000.0,
 'B_EX_glc__D_e': 10,
 'B_EX_zn2_e': 1000.0,
 'B_EX_mg2_e': 1000.0,
 'B_EX_ca2_e': 1000.0,
 'B_EX_ni2_e': 1000.0,
 'B_EX_cu2_e': 1000.0,
 'B_EX_cobalt2_e': 1000.0,
 'B_EX_sel_e': 1000.0,
 'B_EX_h2o_e': 1000.0,
 'B_EX_nh4_e': 1000.0,
 'B_EX_mobd_e': 1000.0,
 'B_EX_so

In [52]:
for i in model.medium:
    h=i.split('_')[0]
    if h=='A' or h=='B':
        model.reactions.get_by_id(i).bounds=(0,0)
model.medium

{'EX_pi_e': 1000.0,
 'EX_h_e': 1000.0,
 'EX_fe3_e': 1000.0,
 'EX_mn2_e': 1000.0,
 'EX_co2_e': 1000.0,
 'EX_fe2_e': 1000.0,
 'EX_glc__D_e': 10.0,
 'EX_xyl__D_e': 10.0,
 'EX_zn2_e': 1000.0,
 'EX_mg2_e': 1000.0,
 'EX_ca2_e': 1000.0,
 'EX_ni2_e': 1000.0,
 'EX_cu2_e': 1000.0,
 'EX_cobalt2_e': 1000.0,
 'EX_sel_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_mobd_e': 1000.0,
 'EX_so4_e': 1000.0,
 'EX_k_e': 1000.0,
 'EX_na1_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_cl_e': 1000.0,
 'EX_tungs_e': 1000.0,
 'EX_slnt_e': 1000.0}

In [53]:
model.add_reaction(model_b.reactions.EX_Salid_e)

In [54]:
model.add_reaction(model_b.reactions.EX_tyr__L_e)
model.add_reaction(model_b.reactions.EX_phpyr_e)

In [55]:
model_b.reactions.EX_tyr__L_e

Reaction identifier,EX_tyr__L_e
Name,L-Tyrosine exchange
Memory address,0x027e9d463978
Stoichiometry,tyr__L_e <-- L-Tyrosine <--
GPR,
Lower bound,-1000
Upper bound,0


In [56]:
model_b.reactions.EX_phpyr_e

Reaction identifier,EX_phpyr_e
Name,phpyr_e exchange
Memory address,0x027ea8fa91d0
Stoichiometry,phpyr_e <=> Phenylpyruvate <=>
GPR,
Lower bound,-1000
Upper bound,1000


In [57]:
#Interaction constraints between sub-models
model.reactions.get_by_id('B_Tyrosol_e_con_2').bounds=(0,1000)
model.reactions.get_by_id('A_Tyrosol_e_con_1').bounds=(0,1000)
model.reactions.get_by_id('B_tyr__L_e_con_2').bounds=(0,1000)
model.reactions.get_by_id('A_tyr__L_e_con_1').bounds=(0,1000)
model.reactions.get_by_id('B_phpyr_e_con_2').bounds=(-1000,0)
model.reactions.get_by_id('A_phpyr_e_con_1').bounds=(-1000,0)

In [58]:
#gene correction
model.reactions.get_by_id('A_ACMANAptspp').bounds=(0,0)
model.reactions.get_by_id('A_MANptspp').bounds=(0,0)
model.reactions.get_by_id('A_FRUpts2pp').bounds=(0,0)
model.reactions.get_by_id('A_GAMptspp').bounds=(0,0)
model.reactions.get_by_id('A_2DGLCptspp').bounds=(0,0)
#
model.reactions.get_by_id('B_XYLI2').bounds=(0,0)
model.reactions.get_by_id('A_HEX1').bounds=(0,0)

In [59]:
model.objective=model.reactions.EX_Salid_e
model.optimize()
model.summary()

IN FLUXES       OUT FLUXES      OBJECTIVES
--------------  --------------  ----------------
o2_e      11.8  h2o_e    46.6   EX_Salid_e  6.34
glc__D_e  10    co2_e    21.3
xyl__D_e  10    Salid_e   6.34


In [60]:
model.reactions.A_BIOMASS_Ec_iML1515_core_75p37M.bounds=(0.1,1000)
model.reactions.B_BIOMASS_Ec_iML1515_core_75p37M.bounds=(0.1,1000)

In [61]:
model.reactions.B_phpyr_e_con_2.bounds=(-1000,1000)
model.reactions.A_tyr__L_e_con_1.bounds=(-1000,1000)

In [62]:
#AG GD excess synthesis
model.reactions.B_EX_phpyr_e.bounds=(1,1000)
model.reactions.A_EX_tyr__L_e.bounds=(1,1000)

In [63]:
save_json_model(model,model_ab_json)

# Simulation

In [64]:
from CulECpyD_radio import *
from concurrent.futures import ProcessPoolExecutor, as_completed
import datetime
import sys
sys.path.append(r'./')

In [65]:
model_file=model_ab_json
# model_file='./iML1515_AGGD_change.json'
reaction_kcat_MW_file='./new_idea_AGGD/EC/reaction_kcat_MW_AGGD_origin.csv'
Concretemodel_Need_Data=Get_Concretemodel_Need_Data_json(model_file,reaction_kcat_MW_file,'Salid_c')
Concretemodel_Need_Data['reaction_kcat_MW']=unpdate_kcat_mw(Concretemodel_Need_Data['reaction_list'],Concretemodel_Need_Data['reaction_kcat_MW'],'./new_idea_AGGD/EC/reaction_kcat_MW_AGGD.csv')

In [66]:
list_x=[9,8,6,4]
list_y=[3,4,6,8]

In [67]:
if __name__ == "__main__":
    pool=ProcessPoolExecutor(max_workers=8)
    with pool as executor:
        start = datetime.datetime.now()
        #print("pass_2")
        tmp=[]
        futures = {executor.submit(diff_subs_double_iML,Concretemodel_Need_Data,(list_x[i],list_y[i]),{'EX_glc__D_e_reverse':93,'EX_xyl__D_e_reverse':27},'EX_Salid_e'): i for i in range(0,len(list_x))}
        # for future in as_completed(futures):
        #     tmp.append(future.result())
    end = datetime.datetime.now()
    print (end - start)
    print(futures)

0:00:19.273466
{<Future at 0x27eb2def0b8 state=finished returned float>: 0, <Future at 0x27eb31b5358 state=finished returned float>: 1, <Future at 0x27eb31b5400 state=finished returned float>: 2, <Future at 0x27eb31b5550 state=finished returned float>: 3}


In [68]:
result_list_mass=pd.DataFrame()
for future in as_completed(futures):
    x=list_x[futures[future]]
    y=list_y[futures[future]]
    real=1
    real_x=int(x*real)
    real_y=int(y*real)
    print('A:B='+str(real_x)+':'+str(real_y))
    print(future.result())
    result_list_mass.at[str(real_x)+':'+str(real_y),'flux']=future.result()
result_list=pd.DataFrame()
for i in range(0,len(list_x)):
    real=1
    real_x=int(list_x[i]*real)
    real_y=int(list_y[i]*real)
    result_list.at[str(real_x)+':'+str(real_y),'flux']=result_list_mass.at[str(real_x)+':'+str(real_y),'flux']
result_list

A:B=8:4
4.444602875792043
A:B=4:8
1.4262606225920536
A:B=9:3
3.969417828304586
A:B=6:6
3.1946479559449017


,flux
9:3,3.969418
8:4,4.444603
6:6,3.194648
4:8,1.426261
